# Data Scraping

Dieses Jupyter Notebook wurde genutzt, um alle Verbindungen im Schweizer ÖV-Betriebsnetz zu scrapen und die Daten in einem Dataframe zu filtern und exportieren. Da die Dateien jeweils mehrere GB speicher benötigten, wurden die gescrapten Dateien direkt gelöscht und nur die relevanten Informationen gespeichert.

In [46]:

# import libraries
import os
import pandas as pd
import requests
import zipfile
import App.helper.config as config
import shutil

In [47]:
# Function to filter the data
def filter_data(data: pd.DataFrame, haltestellen: list, linien: str):
    # find number if NaN values in the HALTESTELLEN_NAME and LINIEN_TEXT columns
    print("Number of NaN values in HALTESTELLEN_NAME column: ", data['HALTESTELLEN_NAME'].isnull().sum())
    print("Number of NaN values in LINIEN_TEXT column: ", data['LINIEN_TEXT'].isnull().sum())
    # drop rows with NaN values in the HALTESTELLEN_NAME and LINIEN_TEXT columns
    data.dropna(subset=['HALTESTELLEN_NAME', 'LINIEN_TEXT'], inplace=True)
    # Filter HALTESTELLEN_NAME by the list of stations
    data = data[data['HALTESTELLEN_NAME'].isin(haltestellen)]
    # Filter LINIEN_TEXT by the list of lines
    data = data[data['LINIEN_TEXT'].str.contains(linien)]
    return data

In [48]:
def merge_stations(df, station_1, station_2):
    # Define columns variable
    columns = ['BETRIEBSTAG', 'LINIEN_ID', 'LINIEN_TEXT']

    # Filter df for station_1 and station_2
    df = df[(df['HALTESTELLEN_NAME'] == station_1) | (df['HALTESTELLEN_NAME'] == station_2)]

    # Create new dataframe with AN_soll, AN_ist, AN_diff if AN_ist_STATUS is "REAL"
    df_an = df[df['AN_ist_STATUS'] == 'REAL'][columns + ['HALTESTELLEN_NAME', 'AN_soll', 'AN_ist', 'AN_diff']]
    # Rename column HALTESTELLEN_NAME to haltestelle_an
    df_an = df_an.rename(columns={'HALTESTELLEN_NAME': 'haltestelle_an'})

    # Create new dataframe with AB_soll, AB_ist, AB_diff if AB_ist_STATUS is "REAL"
    df_ab = df[(df['AB_ist_STATUS'] == 'REAL')][columns + ['HALTESTELLEN_NAME', 'AB_soll', 'AB_ist', 'AB_diff']]
    # Rename column HALTESTELLEN_NAME to haltestelle_ab
    df_ab = df_ab.rename(columns={'HALTESTELLEN_NAME': 'haltestelle_ab'})

    # Merge dataframes df_an and df_ab by BETRIEBSTAG and LINIEN_ID
    df_merged = pd.merge(df_an, df_ab, on=columns)

    # Drop rows with haltestelle_an == haltestelle_ab
    df_merged = df_merged[df_merged['haltestelle_an'] != df_merged['haltestelle_ab']]

    return df_merged

In [49]:
# Funtion to convert data to correct format
def convert_data(df):

    # Rename columns, replace names ending with PROGNOSE with ist
    df.columns = df.columns.str.replace('PROGNOSE', 'ist')
    # Replace ANKUNFTSZEIT with AN_soll und ABFAHRTSZEIT with AB_soll
    df.columns = df.columns.str.replace('ANKUNFTSZEIT', 'AN_soll')
    df.columns = df.columns.str.replace('ABFAHRTSZEIT', 'AB_soll')

    # Create new column AN_diff with difference between AN_ist and AN_soll
    df['AN_diff'] = df['AN_ist'] - df['AN_soll']

    # Create new column AB_diff with difference between AB_ist and AB_soll
    df['AB_diff'] = df['AB_ist'] - df['AB_soll']

    # Find stations
    haltestellen = df['HALTESTELLEN_NAME'].unique()
    
    # Create empty dataframe to store merged data
    df_merged = pd.DataFrame()

    # Apply merge_stations function to all station combinations
    for station_1 in haltestellen:
        for station_2 in haltestellen:
            if station_1 != station_2:
                df_merged = pd.concat([df_merged, merge_stations(df, station_1, station_2)])
                    
    # Create new columns from BETRIEBSTAG: weekday
    df_merged['weekday'] = df_merged['BETRIEBSTAG'].dt.weekday

    # Create column ab_hour and ab_minute from AB_soll
    df_merged['ab_hour'] = df_merged['AB_soll'].dt.hour
    df_merged['ab_minute'] = df_merged['AB_soll'].dt.minute

    # Convert timedelta to minutes
    df_merged['AN_diff'] = df_merged['AN_diff'].dt.total_seconds() / 60
    df_merged['AB_diff'] = df_merged['AB_diff'].dt.total_seconds() / 60

    return df_merged

In [50]:
def process_data(url, data, item):

    # Define the folder to extract the zip file contents
    folder_name = 'Zipped_data'

    # Download the zip file
    r = requests.get(url)
    
    with open(f'ist-daten-{item}.zip', 'wb') as f:
        f.write(r.content)
    
    # Extract the contents of the zip file into the folder
    with zipfile.ZipFile(f'ist-daten-{item}.zip', 'r') as zip_ref:
        zip_ref.extractall(folder_name)
    
    # Read each file in the folder into a DataFrame and store them in a list
    haltestellen = config.HALTESTELLEN 
    # Linien types as string separated by |
    linien = '|'.join(config.LINIEN_TYPES)
    # Define empty list to store dataframes
    df_list = []
    print(f"Processing data...")
    # Loop through all files in the folder
    for file_name in os.listdir(folder_name):
        # Check if file is a csv file
        if file_name.endswith('.csv'):
            print(file_name)
            file_path = os.path.join(folder_name, file_name)
            print(file_path)
            # Read csv file into a DataFrame
            df = pd.read_csv(file_path, delimiter = ';', on_bad_lines='skip', parse_dates=['BETRIEBSTAG', 'ANKUNFTSZEIT', 'ABFAHRTSZEIT', 'AB_PROGNOSE', 'AN_PROGNOSE'], dayfirst=True, low_memory=False)
            # Filter data
            df = filter_data(df, haltestellen, linien)
            # Convert data
            df = convert_data(df)
            # Append DataFrame to list
            df_list.append(df)
            print(f"processed {file_name}.")

    # Concatenate the DataFrames into a single DataFrame
    df_combined = pd.concat(df_list, axis=0)
    
    # Delete the folder and the zip file
    osx_path = os.path.join(folder_name, '__MACOSX')
    if os.path.isdir(osx_path):
        shutil.rmtree(osx_path)
        print(f'Removed directory: {osx_path}')
    else:
        print(f'Directory not found: {osx_path}')
    os.remove(f'ist-daten-{item}.zip')
    
    # Delete all files in folder
    for file_name in os.listdir(folder_name):
        file_path = os.path.join(folder_name, file_name)
        if os.path.isdir(file_path):
            os.rmdir(file_path)
            print(f'if os.path.isdir(file_path):{file_path}')
        else:
            os.remove(file_path)
            print(f'else:{file_path}')
    # Delete folder
    if len(os.listdir(folder_name)) == 0:
        os.rmdir(folder_name)
        print(f'len(os.listdir(folder_name)) == 0{folder_name}')
    else:
        print(f"Unable to delete directory {folder_name} as it still contains files.")
    return df_combined


In [51]:
def process_special_data(url, data, item):

    # Define the folder to extract the zip file contents
    folder_name = 'Zipped_data'
    
    # Download the zip file
    r = requests.get(url)
    
    with open(f'ist-daten-{item}.zip', 'wb') as f:
        f.write(r.content)
    
    # Extract the contents of the zip file into the folder
    with zipfile.ZipFile(f'ist-daten-{item}.zip', 'r') as zip_ref:
        zip_ref.extractall(folder_name)
    
    # Read each file in the folder into a DataFrame and store them in a list
    haltestellen = config.HALTESTELLEN 
    # Linien types as string separated by |
    linien = '|'.join(config.LINIEN_TYPES)

    # Define empty list to store dataframes
    df_list = []
    print(f"Processing data...")
    # Loop through all files in the folder
    for file_name in os.listdir(f'{folder_name}/ist-daten-{item}'):
        # Check if file is a csv file
        if file_name.endswith('.csv'):
            print(file_name)
            file_path = os.path.join(f'{folder_name}/ist-daten-{item}', file_name)
            print(file_path)
            # Read csv file into a DataFrame
            df = pd.read_csv(file_path, delimiter = ';', on_bad_lines='skip', parse_dates=['BETRIEBSTAG', 'ANKUNFTSZEIT', 'ABFAHRTSZEIT', 'AB_PROGNOSE', 'AN_PROGNOSE'], dayfirst=True, low_memory=False)
            # Filter data
            df = filter_data(df, haltestellen, linien)
            # Convert data
            df = convert_data(df)
            # Append DataFrame to list
            df_list.append(df)
            print(f"processed {file_name}.")

    # Concatenate the DataFrames into a single DataFrame
    df_combined = pd.concat(df_list, axis=0)
   
    # Delete the folder and the zip file
    os.remove(f'ist-daten-{item}.zip')
    #Delete manually
    return df_combined


Die erarbeitenen Funtionen werden in diesem Teil ausgeführt.

In [52]:
# Define the folder to extract the zip file contents
scraper_list = ['2023-05', '2023-03', '2023-02', '2023-01','2022-11', '2022-10', '2022-09', '2022-08', '2022-07', '2022-06']
scraper_list += ['2022-05', '2022-04', '2022-03', '2022-02', '2022-01', '2021-12', '2021-11', '2021-10', '2021-09', '2021-08', '2021-07', '2021-06']
# Define empty DataFrame to store the combined data
data = pd.DataFrame()
for item in scraper_list:
    url = f'https://opentransportdata.swiss/wp-content/uploads/ist-daten-archive/ist-daten-{item}.zip'
    data = process_data(url, data, item)
    data.to_csv(f'Daten/v3/combined_data_{item}.csv', index=False)

Processing data...
2023-03-01_istdaten.csv
Zipped_data\2023-03-01_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  162842
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-03-01_istdaten.csv.
2023-03-02_istdaten.csv
Zipped_data\2023-03-02_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  163527
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-03-02_istdaten.csv.
2023-03-03_istdaten.csv
Zipped_data\2023-03-03_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  164654
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-03-03_istdaten.csv.
2023-03-04_istdaten.csv
Zipped_data\2023-03-04_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  135707
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-03-04_istdaten.csv.
2023-03-05_istdaten.csv
Zipped_data\2023-03-05_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  97656
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-03

In [55]:
# für fehlerhafte Dateien wurde ein eigener Prozess erstellt
scraper_list = ['2023-04'] # '2022-12', 
data = pd.DataFrame() # pd.read_csv('combined_data.csv', delimiter=';')
for item in scraper_list:
    url = f'https://opentransportdata.swiss/wp-content/uploads/ist-daten-archive/ist-daten-{item}.zip'
    data = process_special_data(url, data, item)
    data.to_csv(f'Daten/v3/combined_data_{item}.csv', index=False)

Processing data...
2023-04-01_istdaten.csv
Zipped_data/ist-daten-2023-04\2023-04-01_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  134297
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-04-01_istdaten.csv.
2023-04-02_istdaten.csv
Zipped_data/ist-daten-2023-04\2023-04-02_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  95524
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-04-02_istdaten.csv.
2023-04-03_istdaten.csv
Zipped_data/ist-daten-2023-04\2023-04-03_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  159521
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-04-03_istdaten.csv.
2023-04-04_istdaten.csv
Zipped_data/ist-daten-2023-04\2023-04-04_istdaten.csv
Number of NaN values in HALTESTELLEN_NAME column:  160437
Number of NaN values in LINIEN_TEXT column:  0
processed 2023-04-04_istdaten.csv.
2023-04-05_istdaten.csv
Zipped_data/ist-daten-2023-04\2023-04-05_istdaten.csv
Number of NaN values in HALTESTE

Finaler Datensatz mit Informationen wird zusammengestellt und gespeichert für das Feature Engineering.

In [57]:
# Delete the folder and the zip file
for file_name in os.listdir('Zipped_data'):
    file_path = os.path.join('Zipped_data', file_name)
    if os.path.isdir(file_path):
        shutil.rmtree(file_path)
        print(f'Removed directory: {file_path}')
    else:
        os.remove(file_path)
        print(f'Removed file: {file_path}')

Removed directory: Zipped_data\ist-daten-2022-12
Removed directory: Zipped_data\ist-daten-2023-04


In [56]:
# csv aus ordner zusammenfügen
import pandas as pd
import os

# Define the folder to extract the zip file contents
folder_name = 'Daten/v3'
df_list = []
for file_name in os.listdir(folder_name):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_name, file_name)
        df = pd.read_csv(file_path, delimiter = ';', on_bad_lines='skip', dayfirst=True, low_memory=False)
        df_list.append(df)
        print(f"processed {file_name}.")
df_combined = pd.concat(df_list, axis=0)
df_combined.drop_duplicates(inplace=True)
df_combined.to_csv('./Daten/two_years/combined_data.csv',sep=";", index=False)

processed combined_data_2021-06.csv.
processed combined_data_2021-07.csv.
processed combined_data_2021-08.csv.
processed combined_data_2021-09.csv.
processed combined_data_2021-10.csv.
processed combined_data_2021-11.csv.
processed combined_data_2021-12.csv.
processed combined_data_2022-01.csv.
processed combined_data_2022-02.csv.
processed combined_data_2022-03.csv.
processed combined_data_2022-04.csv.
processed combined_data_2022-05.csv.
processed combined_data_2022-06.csv.
processed combined_data_2022-07.csv.
processed combined_data_2022-08.csv.
processed combined_data_2022-09.csv.
processed combined_data_2022-10.csv.
processed combined_data_2022-11.csv.
processed combined_data_2022-12.csv.
processed combined_data_2023-01.csv.
processed combined_data_2023-02.csv.
processed combined_data_2023-03.csv.
processed combined_data_2023-04.csv.
processed combined_data_2023-05.csv.


In [60]:
# Split df_combined in two
df_combined_1 = df_combined.iloc[:int(len(df_combined)/2)]
df_combined_2 = df_combined.iloc[int(len(df_combined)/2):]
# Save to csv
df_combined_1.to_csv('./Daten/two_years/combined_data_1.csv',sep=";", index=False)
df_combined_2.to_csv('./Daten/two_years/combined_data_2.csv',sep=";", index=False)